In [24]:
import pandas as pd
import joblib
import numpy as np
from datetime import timedelta
from datetime import datetime
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [ ]:
def extract_event_features(df):
    """
    연속혈당 데이터에서 혈당 상승 패턴을 기반으로 식사 이벤트를 추정하고,
    각 이벤트에 대해 식전/식후 혈당 및 변화량을 계산하여 반환.
    """
    result_rows = []

    # 파라미터(생리적 곡선 근거, 상황에 따라 조정 가능)
    rise_check_window_min = 20         # 식사 후 혈당상승을 확인할 시간(분)
    rise_check_window = rise_check_window_min // 5  # 데이터 간격이 5분 단위라고 가정
    pre_window_min = 30                # 식전 혈당평균 구간(분)
    rise_thresh = 15                   # 혈당상승폭 기준 (mg/dL)
    min_gap = timedelta(minutes=90)    # 이벤트 간 최소 간격

    # 사용자별 데이터 처리
    for user_id, user_df in df.groupby('ID'):
        user_df = user_df.sort_values('Timestamp').reset_index(drop=True)
        meal_events = []
        last_event_time = None

        # 인덱스 순회하며 식사 이벤트 예상
        for i in range(rise_check_window, len(user_df)):
            # 1) 식전 30분 평균 계산
            pre_start = max(i - (pre_window_min // 5), 0)
            pre_gl = user_df['GL'].iloc[pre_start:i].mean()

            # 2) 식후 20분 내 최대 혈당 찾기
            post_end = min(i + rise_check_window, len(user_df)-1)
            post_max = user_df['GL'].iloc[i:post_end+1].max()

            # 3) 상승폭 계산
            rise = post_max - pre_gl
            cur_time = user_df['Timestamp'].iloc[i]

            # 4) 혈당이 기준 이상 오르고, 이벤트 간 최소 간격을 넘겼다면 식사 이벤트로 추정
            if rise >= rise_thresh:
                if (last_event_time is None) or ((cur_time - last_event_time) >= min_gap):
                    meal_events.append(cur_time)
                    last_event_time = cur_time

        # 추정된 식사 이벤트별로 상세 정보 추출 (1시간 예측에 필요한 부분만)
        for meal_time in meal_events:
            # 5) 식전 30분 평균 (추정 이벤트 시각 기준)
            pre_mask = (user_df['Timestamp'] < meal_time) & \
                       (user_df['Timestamp'] >= meal_time - timedelta(minutes=pre_window_min))
            pre_gl = user_df.loc[pre_mask, 'GL'].mean()

            post_1h_start = meal_time + timedelta(minutes=40)
            post_1h_end = meal_time + timedelta(minutes=80)
            mask_1h = (user_df['Timestamp'] >= post_1h_start) & (user_df['Timestamp'] <= post_1h_end)
            if not user_df.loc[mask_1h].empty:
                post_1h_gl = user_df.loc[mask_1h, 'GL'].max()
            else:
                # 40~80분 구간에 값이 없다면, ±30분 내 가장 가까운 혈당값 사용
                target = meal_time + timedelta(minutes=60)
                window = 30  # ±30분
                mask_any = (user_df['Timestamp'] >= target - timedelta(minutes=window)) & \
                           (user_df['Timestamp'] <= target + timedelta(minutes=window))
                if not user_df.loc[mask_any].empty:
                    idx = (user_df.loc[mask_any, 'Timestamp'] - target).abs().idxmin()
                    post_1h_gl = user_df.loc[idx, 'GL']
                else:
                    post_1h_gl = np.nan

            # 7) 변화량 계산 (1시간 후만)
            delta_gl_1h = post_1h_gl - pre_gl if pd.notnull(pre_gl) and pd.notnull(post_1h_gl) else np.nan

            # 8) 결과 row 저장 (기본정보 + 1시간 파생변수)
            meal_idx = (user_df['Timestamp'] - meal_time).abs().idxmin()
            row = user_df.loc[meal_idx].to_dict()

            row['Meal_Event_Time'] = meal_time
            row['Pre GL'] = pre_gl
            row['Post1h GL'] = post_1h_gl
            row['Delta GL 1h'] = delta_gl_1h

            result_rows.append(row)
    return pd.DataFrame(result_rows)

def calc_auc(trace, time_col, glucose_col):
    """
    혈당 데이터 구간에서 AUC(면적)를 계산하는 함수

    Parameters:
    - trace: 특정 시간 구간의 DataFrame (예: 1시간 동안의 Timestamp, GL 데이터)
    - time_col: 시간 컬럼명 (예: 'Timestamp')
    - glucose_col: 혈당 컬럼명 (예: 'GL')

    Returns:
    - 해당 시간 구간의 혈당 곡선 아래 면적 (단위: mg/dL x 분)
    """
    if trace.empty or trace.shape[0] < 2: # 데이터가 없거나 2개 미만이면 AUC 계산 불가
        return np.nan
    # 시간 차이를 분 단위로 변환
    time_min = (trace[time_col] - trace[time_col].min()).dt.total_seconds() / 60
    # 혈당 값 추출
    glucose = trace[glucose_col].values
    auc = np.trapz(glucose, time_min)
    return auc

def add_auc_features(df_event, base_df, interval_minutes_1h=60):
    """
    이벤트 기반 DataFrame에 AUC 및 보정 AUC 파생 피처를 추가하는 함수

    Parameters:
    - df_event: 식사 이벤트 정보가 들어 있는 DataFrame
    - base_df: 원본 혈당 데이터 전체 (ID, Timestamp, GL 포함)
    - interval_minutes_1h: AUC 계산할 구간 길이 (기본값: 60분)

    Returns:
    - AUC 및 공복혈당 보정 AUC가 추가된 df_event 반환
    """
    aucs_1h = []
    for idx, row in df_event.iterrows():
        user_id = row['ID']
        meal_time = row['Meal_Event_Time']
        user_df = base_df[base_df['ID'] == user_id]

        start_1h = meal_time
        end_1h = meal_time + timedelta(minutes=interval_minutes_1h)
        mask_1h = (user_df['Timestamp'] >= start_1h) & (user_df['Timestamp'] <= end_1h)
        trace_1h = user_df.loc[mask_1h, ['Timestamp', 'GL']].copy()
        auc_1h = calc_auc(trace_1h, 'Timestamp', 'GL')
        aucs_1h.append(auc_1h)

    df_event['AUC_1h'] = aucs_1h
    df_event['AUC_adj_1h'] = df_event['AUC_1h'] - (df_event['Pre GL'] * interval_minutes_1h)
    return df_event


def predict_for_all(model, user_file_path):
    # 1. 전체 데이터 로드
    df = pd.read_csv(user_file_path)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

    # 2. 전체 데이터에서 이벤트/피처 추출
    df_event = extract_event_features(df)
    df_event = add_auc_features(df_event, df)

    # 3. Gender 변환
    if df_event['Gender'].dtype == 'object':
        df_event['Gender'] = df_event['Gender'].map({'M':1, 'F':0, '남':1, '여':0})
    df_event['Gender'] = df_event['Gender'].astype(float)

    # 4. feature만 남기고 결측치 제거
    feature_cols = [ 'Time of Day', 'Age', 'Gender', 'Weight(kg)', 'Height(cm)', 'GL', 'AUC_adj_1h' ]
    df_event = df_event.dropna(subset=feature_cols + ['Delta GL 1h'])  # 예측값, 실제값 둘다 있는 것만
    X = df_event[feature_cols].astype(float)

    # 5. 예측
    pred = model.predict(X)
    df_event['Predicted Delta GL 1h'] = pred

    # 6. 평가 지표
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    y_true = df_event['Delta GL 1h']
    mae = mean_absolute_error(y_true, pred)
    rmse = np.sqrt(mean_squared_error(y_true, pred))
    r2 = r2_score(y_true, pred)
    print(f"[전체 데이터] MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.3f}")

    # 7. 결과 반환
    return df_event[['ID', 'Timestamp', 'Delta GL 1h', 'Predicted Delta GL 1h']]

# 모델 불러오기
model_1h = joblib.load("xgboost_1h_model.joblib")

# 실제 데이터 파일 이름
user_file = "CGMNDYoungChildren_final.csv"

result_df = predict_for_all(model_1h, user_file)


C:\Users\wjdek\AppData\Local\Temp\ipykernel_19796\569456544.py:81: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  auc = np.trapz(glucose, time_min)
C:\Users\wjdek\AppData\Local\Temp\ipykernel_19796\569456544.py:81: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  auc = np.trapz(glucose, time_min)
C:\Users\wjdek\AppData\Local\Temp\ipykernel_19796\569456544.py:81: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  auc = np.trapz(glucose, time_min)
C:\Users\wjdek\AppData\Local\Temp\ipykernel_19796\569456544.py:81: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  auc = np.trapz(glucose, time_min)
C:\Users\wjdek\AppData\Local\Temp\ipykernel_

[전체 데이터] MAE: 10.60, RMSE: 15.91, R²: 0.592


C:\Users\wjdek\AppData\Local\Temp\ipykernel_19796\569456544.py:81: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  auc = np.trapz(glucose, time_min)
C:\Users\wjdek\AppData\Local\Temp\ipykernel_19796\569456544.py:81: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  auc = np.trapz(glucose, time_min)
C:\Users\wjdek\AppData\Local\Temp\ipykernel_19796\569456544.py:81: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  auc = np.trapz(glucose, time_min)
C:\Users\wjdek\AppData\Local\Temp\ipykernel_19796\569456544.py:81: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  auc = np.trapz(glucose, time_min)
C:\Users\wjdek\AppData\Local\Temp\ipykernel_

In [22]:
# 1. 모델 및 데이터 로드
model = joblib.load('xgboost_1h_model.joblib')
user_file = "CGMNDYoungChildren_final.csv"
result_df = predict_for_all(model, user_file)

# 2. 개인별 평균 예측 변화량 기반 탄수화물 upper bound 추천 함수
def map_change_to_carb(delta):
    if delta <= 10:
        return (60, 100)
    elif delta <= 15:
        return (40, 60)
    elif delta <= 20:
        return (20, 40)
    else:
        return (10, 20)

# 3. ID별 추천 테이블 만들기
recommend_df = (
    result_df.groupby('ID')['Predicted Delta GL 1h']
    .mean().reset_index(name='mean_pred_delta')
)
recommend_df['carb_range'] = recommend_df['mean_pred_delta'].apply(map_change_to_carb)

# 4. 프롬프트로 아이디 입력 → 추천 범위 출력
user_id = input("아이디를 입력하십시오: ")
try:
    user_id = int(user_id)
except ValueError:
    print("숫자형 아이디만 입력하세요.")
    exit()

user_row = recommend_df[recommend_df['ID'] == user_id]
if user_row.empty:
    print("존재하지 않는 아이디입니다.")
else:
    carb_range = user_row['carb_range'].values[0]
    print(f"{user_id}번 아이의 추천 탄수화물 허용 범위: {carb_range[0]}~{carb_range[1]}g (예측 평균 변화량: {user_row['mean_pred_delta'].values[0]:.2f} mg/dL)")


C:\Users\wjdek\AppData\Local\Temp\ipykernel_19796\569456544.py:81: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  auc = np.trapz(glucose, time_min)
C:\Users\wjdek\AppData\Local\Temp\ipykernel_19796\569456544.py:81: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  auc = np.trapz(glucose, time_min)
C:\Users\wjdek\AppData\Local\Temp\ipykernel_19796\569456544.py:81: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  auc = np.trapz(glucose, time_min)
C:\Users\wjdek\AppData\Local\Temp\ipykernel_19796\569456544.py:81: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  auc = np.trapz(glucose, time_min)
C:\Users\wjdek\AppData\Local\Temp\ipykernel_

[전체 데이터] MAE: 10.60, RMSE: 15.91, R²: 0.592
1번 아이의 추천 탄수화물 허용 범위: 10~20g (예측 평균 변화량: 28.51 mg/dL)


In [25]:
df = pd.read_csv("조리식품의 레시피 DB.csv", encoding='CP949')
print('데이터 shape:', df.shape)

# 필터링할 컬럼 목록
columns_to_keep = ["RCP_NM", "RCP_PAT2", "INFO_CAR", "INFO_NA", "RCP_PARTS_DTLS"]
filtered_df = df[columns_to_keep]

# 칼럼명 바꾸기
filtered_df = filtered_df.rename(columns={
    "RCP_NM": "음식명",
    "RCP_PAT2": "요리종류",
    "INFO_CAR": "탄수화물",
    "INFO_NA": "나트륨",
    "RCP_PARTS_DTLS": "식재료"
})

filtered_df.head()

# 재료 정제 함수
def clean_ingredient_info(text):
    text = re.sub(r'\d+(\.\d+)?(g|ml|개|컵|큰술|작은술|마리|장|쪽|스푼)?', '', text) # 숫자 제거&단위 제거
    text = re.sub(r'[a-zA-Z]', '', text) # 영어 제거
    text = re.sub(r'\([^)]*\)', '', text) # 괄호 제거
    text = re.sub(r'\[[^\]]*\]', '', text) # 대괄호 제거
    text = re.sub(r'[^\w\s가-힣]', '', text) # 특수문자 제거
    text = re.sub(r'\s+', ' ', text).strip() # 슬래시 이후 제거
    text = text.split()
    return text

filtered_df['식재료'] = filtered_df['식재료'].fillna('').apply(clean_ingredient_info)
filtered_df.head()

데이터 shape: (1136, 55)


,음식명,요리종류,탄수화물,나트륨,식재료
0,새우 두부 계란찜,반찬,3.0,99.0,"[새우두부계란찜, 연두부, 칵테일새우, 달걀, 생크림, 설탕, 무염버터, 고명, 시금치]"
1,부추 콩가루 찜,반찬,20.0,240.0,"[조선부추, 날콩가루, 양념장, 저염간장, 다진, 대파, 다진, 마늘, 고춧가루, ..."
2,방울토마토 소박이,반찬,9.0,277.0,"[방울토마토, 소박이, 방울토마토, 양파, 부추, 양념장, 고춧가루, 멸치액젓, 다..."
3,순두부 사과 소스 오이무침,반찬,10.0,22.0,"[오이무침, 오이, 다진, 땅콩, 순두부사과, 소스, 순두부, 사과]"
4,사과 새우 북엇국,국&찌개,2.0,78.0,"[북엇국, 북어채, 새우, 사과, 양파, 표고버섯, 물]"


In [26]:
# 사용자 입력
# 식재료 제한 및 선호
restricted_input = input("제한할 재료를 입력하세요: ")
restricted_ingredients = restricted_input.strip().split() if restricted_input.strip() else []

preferred_input = input("선호하는 재료를 입력하세요: ") 
preferred_ingredients = preferred_input.strip().split() if preferred_input.strip() else []

# 비선호 재료 필터링
def filter_restricted_foods(df, restricted_ingredients):
    # 제한 식재료 제거
    for item in restricted_ingredients:
        df = df[~df["식재료"].apply(lambda x: any(item in ing for ing in x))]
    return df

# 선호 재료 및 나트륨 정렬
def sort_by_preference_and_sodium(df, preferred_ingredients):
    # 선호 재료 개수 점수 계산
    df["선호도"] = df["식재료"].apply(lambda x: sum(1 for pref in preferred_ingredients if pref in x))
    # 나트륨 정제 (숫자형 변환)
    df["나트륨"] = df["나트륨"].astype(float)
    df = df.sort_values(by=["선호도", "나트륨"], ascending=[False, True])
    return df

removed_df = filter_restricted_foods(filtered_df, restricted_ingredients)
sorted_df = sort_by_preference_and_sodium(removed_df, preferred_ingredients)

C:\Users\wjdek\AppData\Local\Temp\ipykernel_19796\3481940971.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["선호도"] = df["식재료"].apply(lambda x: sum(1 for pref in preferred_ingredients if pref in x))
C:\Users\wjdek\AppData\Local\Temp\ipykernel_19796\3481940971.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["나트륨"] = df["나트륨"].astype(float)


In [27]:
# 탄수화물 범위 내에서 식단 조합 추천
def recommend_meal(df, carb_min, carb_range):
    # 요리 종류별 대표 음식 추천 (탄수화물 내림차순)
    main = df[df["요리종류"].isin(["밥", "일품"])].head(10)
    soup = df[df["요리종류"] == "국&찌개"].head(5)
    side = df[df["요리종류"] == "반찬"].head(5)
    dessert = df[df["요리종류"] == "후식"].head(5)

    # 조합 만들기
    combinations = []
    for m in main.itertuples():
        m_car = float(m.탄수화물)
        for s in [None] + list(soup.itertuples()):
            s_car = float(s.탄수화물) if s else 0
            for sd in [None] + list(side.itertuples()):
                sd_car = float(sd.탄수화물) if sd else 0
                for d in [None] + list(dessert.itertuples()):
                    d_car = float(d.탄수화물) if d else 0
                    total_car = m_car + s_car + sd_car + d_car
                    
                    if carb_min <= total_car <= carb_range:
                        combinations.append({
                            "주요리": m.음식명,
                            "국": s.음식명 if s else "",
                            "반찬": sd.음식명 if sd else "",
                            "후식": d.음식명 if d else "",
                            "총탄수화물": total_car,
                        })

    return pd.DataFrame(combinations).sort_values(by="총탄수화물", ascending=True).head()

# 추천 식단 도출
recommended_meal_df = recommend_meal(sorted_df, 15, 50) #carb_min #carb_max

# 출력
print("추천 식단 조합:")
recommended_meal_df.head()

추천 식단 조합:


,주요리,국,반찬,후식,총탄수화물
355,클럽 샌드위치,배추된장국,고등어 강정,누룽지 요거트 파르페,15.1
354,클럽 샌드위치,배추된장국,고등어 강정,,15.1
250,클럽 샌드위치,,고등어구이,잣 호두 강정,15.2
329,클럽 샌드위치,롤 삼계탕,고등어 강정,누룽지 요거트 파르페,15.4
328,클럽 샌드위치,롤 삼계탕,고등어 강정,,15.4
